In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [15]:
# need this cell to import the help functions:

import os
path_sections = os.getcwd().split('\\')
if path_sections[0] != 'CGM':
    os.chdir('./..')

In [16]:
# import help functions:

from data_processing.cgm_data import CGMData
from data_processing.cgm_data_helper import plot_series_with_food
from data_processing.derived_datasets import max_glucose_dataset,max_glucose_between_meals_dataset

from sklearn.linear_model import LinearRegression

In [18]:
data = CGMData()

In [21]:
# Finding 5/95th percentile idea from Dan Visscher

high_glu = []
low_glu = []

# Find 5th/95th percentile for glucose range for each patient
for i in range(1,3):
    low_glu.append(np.percentile(data[i].glu, 5))
    high_glu.append(np.percentile(data[i].glu, 95))




In [23]:
print(high_glu)
print(low_glu)

[137.0, 167.0]
[85.0, 99.0]


In [28]:
# Select participant number
participant_number = 1

# Max glucose between meals:
df = max_glucose_dataset(cgm_data=data,participant_num=participant_number,glu_thresh=high_glu[participant_number-1]).head(30)
df


,calorie,total_carb,dietary_fiber,sugar,protein,total_fat,recent_meals,participant,max_glucose,high_glucose
0,456.0,85.0,1.7,83.0,16.0,3.3,1.0,1,143,True
1,488.0,2.5,1.2,0.8,63.4,23.1,1.0,1,106,False
2,230.0,35.0,NaN,18.0,13.0,NaN,1.0,1,122,False
3,280.0,30.0,NaN,22.0,4.0,NaN,1.0,1,110,False
4,358.0,14.4,NaN,8.7,13.9,NaN,1.0,1,124,False
5,308.0,69.0,NaN,38.0,7.2,NaN,1.0,1,102,False
6,418.0,37.2,NaN,11.5,42.0,NaN,1.0,1,114,False
7,12.0,3.4,NaN,0.0,0.2,NaN,1.0,1,142,True
8,158.0,33.0,NaN,13.0,4.0,NaN,1.0,1,124,False
9,227.0,28.2,NaN,17.0,7.5,NaN,1.0,1,119,False


In [ ]:

food5 = data[5].food
glu5 = pd.read_csv('data/005/Dexcom_005.csv', header=0, skiprows=range(1, 13), index_col=['Timestamp (YYYY-MM-DDThh:mm:ss)'], parse_dates=['Timestamp (YYYY-MM-DDThh:mm:ss)'])

plot_series_with_food(glu5, food5, '2020-02-27', '2020-02-29', 'Glucose Value (mg/dL)')

data[5].glu


In [ ]:
X = df[['total_carb','sugar','protein']]
y = df[['high_glucose']]
y

In [ ]:
## Make the model object

reg = LinearRegression()

## Fit the model object

reg.fit(X, y)
reg.coef_, reg.intercept_

In [ ]:
plt.figure(figsize=(6,5))

plt.scatter(df['total_carb'], df['high_glucose'])

plt.xlabel("total_carb", fontsize=12)
plt.ylabel("high_glucose", fontsize=12)
plt.show()

In [ ]:
log_reg = LogisticRegression(penalty=None)
X = df[['total_carb']]
y = df['high_glucose']

## fit the model
log_reg.fit(X,y)

In [ ]:
log_reg.coef_[0][0]

In [ ]:
plt.figure(figsize=(6,5))

plt.scatter(X, y)

plt.plot(np.linspace(0,160), log_reg.predict_proba(np.linspace(0,160).reshape(-1,1))[:,1], color = 'orange')
#plt.plot(np.linspace(7,13), my_log_reg.predict_proba(np.linspace(7,13).reshape(-1,1)), c = 'k')

plt.xlabel("total_carb", fontsize=12)
plt.ylabel("high_glucose", fontsize=12)
plt.show()